In [1]:
import datetime
import sys

sys.path.append("..")

from src.auth.aurora import get_sales_client, get_portal_client

In [2]:
client_name = "A.P.E. Fire & Security"

In [3]:
portal_client = get_portal_client()

In [4]:
root_client_id = portal_client.query_single_value(
    """SELECT id
    FROM api_clients
    WHERE name = %(client_name)s
    AND deleted_at IS NULL
    AND parent_id = 1""",
    {"client_name": client_name},
)

In [5]:
client_list = portal_client.query_to_list(
    """WITH RECURSIVE category_path AS (
    SELECT id, name, parent_id
    FROM api_clients
    WHERE parent_id = %(root_client_id)s
    AND deleted_at IS NULL

    UNION ALL

    SELECT c.id, c.name, c.parent_id
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    WHERE deleted_at IS NULL
    )
    SELECT DISTINCT id
    FROM category_path
    """,
    {"root_client_id": root_client_id},
)
client_list.append(root_client_id)

In [7]:
placeholders = ', '.join(['%s'] * len(client_list))

collabs_df = portal_client.query_to_pandas(
    f"""SELECT *
    FROM topaz.api_collaborators
    WHERE client_id IN ({placeholders})""",
    client_list
)

In [6]:
placeholders = ', '.join(['%s'] * len(client_list))
portal_devices_df = portal_client.query_to_pandas(
    f"""SELECT
        d.id,
        d.device_serial,
        d.imei,
        d_t.type,
        d.client_id,
        d.active,
        MIN(s.created_at) AS created_at,
        MAX(s.expires_at) AS expires_at
    FROM api_devices AS d
    LEFT JOIN api_subscriptions AS s
    ON s.id = d.subscription_id
    LEFT JOIN api_device_types AS d_t
    ON d.device_type_id = d_t.id
    WHERE d.client_id IN ({placeholders})
    AND d.device_type_id IN (0, 1, 14, 15)
    AND device_serial IS NOT NULL
    AND s.created_at IS NOT NULL
    GROUP BY d.id, d.device_serial, d_t.type, d.client_id""",
    params=client_list
)

In [7]:
portal_devices_df[
    (portal_devices_df["expires_at"] > datetime.datetime.now())
    & (portal_devices_df["active"] == 1)
]

,id,device_serial,imei,type,client_id,active,created_at,expires_at
7,22219,G01-00954,0a10aced202194944a051670,Nimbus Go,33,1,2024-11-26,2025-12-31 23:59:59
8,22220,G01-00955,0a10aced202194944a051540,Nimbus Go,33,1,2024-11-26,2025-12-31 23:59:59
9,24063,43090,352890064929837,Nimbus Gateway,33,1,2025-03-10,2026-03-31 23:59:59
10,25492,C01-00807,0a10aced202194944a0756a8,Nimbus Connect,33,1,2025-06-04,2026-06-30 23:59:59
11,25493,C01-00808,0a10aced202194944a0756e0,Nimbus Connect,33,1,2025-06-04,2026-06-30 23:59:59
...,...,...,...,...,...,...,...,...
133,23307,43035,352890064924499,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
134,23308,43036,352890064918392,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
135,23314,43037,352890064912536,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59
136,23315,43038,352890064914631,Nimbus Gateway,24532,1,2025-01-23,2026-01-31 23:59:59


In [8]:
portal_devices_df.to_excel(
    "data/test.xlsx",
    index=False,
)

In [9]:
portal_devices = set(
    portal_devices_df[(
        portal_devices_df["created_at"] <= datetime.datetime.now()
    ) & (
        portal_devices_df["expires_at"] > datetime.datetime.now()
    )]["device_serial"].unique()
)

In [10]:
portal_sites_df = portal_client.query_to_pandas(
    f"""SELECT *
    FROM topaz.api_sites
    WHERE client_id IN ({placeholders})
    AND (
        panel_model_id IS NOT NULL
        OR protocol_id = 37
    )""",
    params=client_list
)

In [11]:
portal_sites_df

,id,urlkey,device_id,protocol_id,panel_model_id,panel_model_other,toc_id,template_id,timezone_id,reference,...,verification_devices,verification_single_zone,responsible_user_id,created_at,updated_at,deleted_at,client_id,hidden,rc_enabled,demo_panel_id
0,141,g3r$z6jwx4,122.0,16,66.0,,204783.0,NaN,36,,...,2,0,NaN,2015-02-02 15:06:59,2025-06-23 07:48:25,NaT,50,0,0,0
1,142,p4n$y68k58,123.0,16,66.0,,170691.0,NaN,36,,...,2,0,NaN,2015-02-02 15:10:25,2025-06-23 08:00:38,NaT,50,0,0,0
2,279,8v0$-95$j3,239.0,16,66.0,,170692.0,NaN,36,,...,2,0,NaN,2015-09-29 12:27:55,2025-06-23 08:01:36,NaT,50,0,0,0
3,280,25mwl31w7x,238.0,16,66.0,,170693.0,NaN,36,,...,2,0,252.0,2015-09-29 13:06:49,2025-06-23 07:59:01,NaT,50,0,0,0
4,281,y43$jv-$gz,240.0,16,66.0,,170694.0,NaN,36,,...,2,0,NaN,2015-09-29 13:07:41,2025-06-23 07:58:15,NaT,50,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,60191,j0knz2jjk_,NaN,5,49.0,,214708.0,NaN,36,0000104305,...,2,0,17469.0,2025-05-23 08:40:35,2025-05-23 11:02:39,NaT,32142,0,0,0
172,60243,v-$95q5ek3,25223.0,1,3.0,,214950.0,3.0,36,Fire Panel,...,2,0,NaN,2025-05-26 10:27:00,2025-06-23 08:01:52,NaT,32184,0,0,0
173,60248,6v$v_2_qw5,NaN,23,68.0,,215084.0,NaN,36,Laganside,...,2,0,NaN,2025-05-27 08:08:22,2025-06-20 06:58:02,NaT,32191,0,0,0
174,60391,j0knz2nlk_,NaN,5,49.0,,215585.0,NaN,36,0000118815,...,2,0,17469.0,2025-05-29 10:48:51,2025-05-29 13:09:10,NaT,32267,0,0,0


In [12]:
disconnected_mask = portal_sites_df["device_id"].isna()
deleted_mask = ~portal_sites_df["deleted_at"].isna()

In [17]:
client_name = "ADT Fire & Security Ltd"
sales_client = get_sales_client()
sales_client_id = sales_client.query_single_value(
    """SELECT client_id
    FROM sales.t_client
    WHERE name = %(client_name)s""",
    {"client_name": client_name},
)

In [18]:
subscriptions_df = sales_client.query_to_pandas(
    """SELECT
        device_id,
        subscription_type,
        item_description,
        start_date,
        expiry_date,
        contract_term,
        (
                CASE WHEN pay_frequency = "Monthly" THEN unit_price
                ELSE unit_price / contract_term
                END
        ) AS mrr
    FROM sales.t_subscription
    LEFT JOIN sales.t_orders
    USING(order_number)
    LEFT JOIN sales.t_item
    USING(item_code)
    WHERE client_id = %(sales_client_id)s""",
    {"sales_client_id": sales_client_id},
)

In [19]:
sales_devices = set(subscriptions_df[(
    (subscriptions_df["start_date"] <= datetime.date.today())
    & (subscriptions_df["expiry_date"] >= datetime.date.today())
)]["device_id"].unique())

In [21]:
sales_devices

{'00128',
 '00129',
 '00138',
 '00173',
 '00174',
 '00175',
 '00253',
 '00314',
 '00319',
 '00320',
 '00321',
 '00366',
 '00367',
 '00376',
 '00437',
 '00444',
 '00454',
 '30012',
 '30013',
 '40123',
 '40339',
 '40369',
 '40492',
 '40730',
 '40816',
 '40817',
 '40980',
 '41051',
 '41055',
 '41339',
 '41413',
 '41414',
 '41489',
 '41615',
 '41616',
 '41617',
 '41618',
 '41633',
 '41653',
 '41684',
 '41759',
 '41841',
 '41848',
 '41864',
 '41871',
 '42082',
 '42088',
 '42089',
 '42100',
 '42161',
 '42162',
 '42163',
 '42164',
 '42165',
 '42166',
 '42167',
 '42168',
 '42169',
 '42298',
 '42394',
 '42395',
 '42396',
 '42460',
 '42570',
 '42571',
 '42572',
 '42573',
 '42574',
 '42575',
 '42576',
 '42577',
 '42578',
 '42579',
 '42580',
 '42581',
 '42582',
 '42583',
 '42584',
 '42585',
 '42586',
 '42587',
 '42588',
 '42589',
 '42590',
 '42591',
 '42592',
 '42593',
 '42594',
 '42595',
 '42596',
 '42597',
 '42598',
 '42599',
 '42600',
 '42601',
 '42602',
 '42603',
 '42604',
 '42606',
 '42607',


In [20]:
subscriptions_df[(
    (subscriptions_df["start_date"] <= datetime.date.today())
    & (subscriptions_df["expiry_date"] >= datetime.date.today())
)]["mrr"].sum()

Decimal('8277.083333')